This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [3]:
import os # Miscellaneous operating system interfaces, provides a portable way of using operating system dependent functionality
import numpy as np # Numerical Python, to work with large, multi-dimensional arrays and matrices,high-level mathematical functions to operate on arrays
from collections import Counter #dict subclass for counting hashable objects
from sklearn.naive_bayes import GaussianNB #implements the Gaussian Naive Bayes algorithm for classification
from sklearn.metrics import accuracy_score  #multilabel classification, computes subset accuracy

This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [5]:
#Function that reads the email files from a folder and constructs a dictionary for all words

def make_dict(main_dir): # defining the function, with parameter "main_dir"
  all_words = [] # creating empty list 
  emails = [os.path.join(main_dir,f) for f in os.listdir(main_dir)] # creating a list 'emails'
  
  # os.path.join() method returns a string which represents the concatenated path components
  # os.listdir() method retrieves a list of all the files in 'main_dir'
  # for loop through all emails in emails
    
  for mail in emails:
    with open(mail) as p: #'with' simplifies management of file streams, no need to call file.close(), save as 'p'
      for line in p:  # loop through each item in email 
        words = line.split()  # returns a list of all the words in the string
        all_words += words # creating list with all words (addition)
  dictionary = Counter(all_words)  # creates dictionary with key + value (keeps track of how many times equivalent values are added)
  list_to_remove = list(dictionary) # returns a list from dictionary 

  for item in list_to_remove: # iterating over the list 
    if item.isalpha() == False:  # deleting if not alphabet letter (a-z), isaplha()- boolean True/False
      del dictionary[item]
    elif len(item) == 1: # deleting if only 1 character (lenght of word = 1)
      del dictionary[item]
  dictionary = dictionary.most_common(3000)  # returns list,sorted based on the count of the elements, limit 3000 and saving back in dictionary
  return dictionary
            

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [9]:
# extract features function that removes less common words from every document 

def extract_feat(email_dir): # defining fucntion with parameter 'email_dir'
  documents = [os.path.join(email_dir,fi) for fi in os.listdir(email_dir)]   # create list with joined path components
  features_matrix = np.zeros((len(documents),3000))  # Return a new array (Matrix) of given shape(len of documents rows) and columns 3000 (and optional type), filled with zeros
  train_labels = np.zeros(len(documents)) # zeros() function --> matrix with 0 with give shape (len of files)
  count = 1;  # start of counter set to 1
  docID = 0; #docId set to 0 
  for fil in documents: # iterate over each file 
    with open(fil) as fi: # save as fi 
      for i, line in enumerate(fi):  # enumerate to get counter in loop
        if i ==2:
          words = line.split() # split if 2 
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')  # use split method at each character backslash ('/') character 
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):  #  we know that spmsga162.txt files are spam 
        train_labels[docID] = 1;  # label as 1 (spam mail), dummy variable?!
        count = count + 1  # increaese by 1 for each loop
      docID = docID + 1
  return features_matrix, train_labels                

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [10]:
TRAIN_DIR = '/Users/maxkaiser/Desktop/train-mails' # file path, download local 
TEST_DIR = '/Users/maxkaiser/Desktop/test-mails' 

In [11]:
dictionary = make_dict(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_feat(TRAIN_DIR)
test_features_matrix, test_labels = extract_feat(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
# train model
model.fit(features_matrix, labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
#predict/test
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))
# percentage of correct predictions 

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154


======================= END OF PROGRAM =========================